<a href="https://colab.research.google.com/github/CinthiaDayanara/modelos_emociones/blob/main/modelodesdecero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install tensorflow
!pip install --upgrade tf-keras
!pip install keras

In [4]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.18.0


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [6]:
# Directorio donde guardar el modelo
model_directory = '/content/drive/MyDrive/model_directory'

In [7]:
# Ruta del dataset

data_dir= '/content/drive/MyDrive/emotions'  # Cambia esta ruta según tu dataset


In [14]:
# Crear un generador de imágenes con aumento de datos (data augmentation) y división de datos en entrenamiento y validación
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalizar las imágenes
    rotation_range=20,  # Rotar aleatoriamente las imágenes
    width_shift_range=0.2,  # Desplazar las imágenes horizontalmente
    height_shift_range=0.2,  # Desplazar las imágenes verticalmente
    shear_range=0.2,  # Aplicar corte aleatorio
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Voltear imágenes horizontalmente
    fill_mode='nearest',  # Rellenar los píxeles vacíos con el valor más cercano
    validation_split=0.2  # 20% de los datos serán para validación
)

# Generador de datos de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,  # Directorio donde están las imágenes
    target_size=(150, 150),  # Tamaño de las imágenes
    batch_size=32,  # Número de imágenes por lote
    class_mode='categorical',  # Clasificación multiclase
    subset='training'  # Usar el 80% de los datos para entrenamiento
)
# Generador de datos de validación
validation_generator = datagen.flow_from_directory(
    data_dir,  # Mismo directorio para la validación
    target_size=(150, 150),  # Tamaño de las imágenes
    batch_size=32,  # Número de imágenes por lote
    class_mode='categorical',  # Clasificación multiclase
    subset='validation'  # Usar el 20% de los datos para validación
)


Found 5326 images belonging to 6 classes.
Found 1330 images belonging to 6 classes.


In [15]:
# Definir el modelo CNN desde cero
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Número de clases de emociones
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
# Resumen del modelo
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,036,742 (72.62 MB)

 Trainable params: 19,036,742 (72.62 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=20,  # Número de épocas para entrenar
    validation_data=validation_generator,  # Datos de validación
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Número de pasos por época
    validation_steps=validation_generator.samples // validation_generator.batch_size  # Número de pasos de validación
)

Epoch 1/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 71s 398ms/step - accuracy: 0.2743 - loss: 2.0111 - val_accuracy: 0.2889 - val_loss: 1.7274
Epoch 2/20
  1/166 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.2812 - loss: 1.7365

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


166/166 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.2812 - loss: 1.7365 - val_accuracy: 0.3333 - val_loss: 1.8045
Epoch 3/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 68s 364ms/step - accuracy: 0.3666 - loss: 1.6036 - val_accuracy: 0.2767 - val_loss: 1.7350
Epoch 4/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.3125 - loss: 1.5478 - val_accuracy: 0.3333 - val_loss: 1.7689
Epoch 5/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 62s 360ms/step - accuracy: 0.3764 - loss: 1.5559 - val_accuracy: 0.2378 - val_loss: 1.8813
Epoch 6/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.4688 - loss: 1.4070 - val_accuracy: 0.1667 - val_loss: 1.9937
Epoch 7/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 62s 364ms/step - accuracy: 0.3905 - loss: 1.5195 - val_accuracy: 0.2386 - val_loss: 1.8094
Epoch 8/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.4062 - loss: 1.4725 - val_accuracy: 0.1111 - val_loss: 2.0832
Epoch 9/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 72s 354ms/step - accuracy: 0.4211 - loss: 1.4823 - val_accura

In [20]:

# Guardar el modelo entrenado
model.save('emotion_model_vgg16_from_scratch.h5')

In [1]:
!pip install tensorflowjs

In [19]:
# Evaluar el modelo en el conjunto de validación
loss, accuracy = model.evaluate(validation_generator)
print(f"Precisión en validación: {accuracy * 100:.2f}%")


42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 280ms/step - accuracy: 0.2443 - loss: 1.7934
Precisión en validación: 24.29%
